In [1]:
import numpy as np
import datetime
import pandas as pd
from datetime import datetime
import vaex as vx
import math


In [2]:
import matplotlib.pyplot as plt
from  plotly.offline import iplot
import plotly as py 
import plotly.express as px

In [3]:
import cufflinks as cf 
py.offline.init_notebook_mode(connected=True)
cf.go_offline()

In [4]:
def convert_to_datetime(date_string):
    return numpy.datetime64(datetime.strptime(str(date_string), '%d/%m/%Y %H:%M:%S'))

def convert_to_date(date_string):
    return numpy.datetime64(datetime.strptime(str(date_string), '%d/%m/%Y'))

def check_wekend(date_string):
    a = datetime.strptime(str(date_string), '%d/%m/%Y')
    weekno = a.weekday()
    if weekno < 5:
        return "Weekday"
    else:  # 5 Sat, 6 Sun
        return "Weekend"
    
def check_month(i):
    if i <= 10:
        return "July-Oct"
    else:
        return "Nov-Dec"
    
def run_preprocess(data):
    data['date_time']  = (data['TICKET DATE'] + ' ' + data['TICKET TIME']).apply(convert_to_datetime)
    data['ticket_date']  = data['TICKET DATE'].apply(convert_to_date)
    data["Day"] = data["date_time"].dt.day_name
    data["Hour"] = data["date_time"].dt.hour
    data["Month"] = data["date_time"].dt.month
    data['day_type'] = data['TICKET DATE'].apply(check_wekend)
    data['Period'] = data['Month'].apply(check_month)
    
def TO_latlong(df):
    to_lat_list = list(df['From_lat'].values[1:])
    to_lat_list.append(to_lat_list[-1])

    to_long_list = list(df['From_lon'].values[1:])
    to_long_list.append(to_long_list[-1])
    
    df['To_lat'] = to_lat_list
    df['To_long'] = to_long_list

In [5]:
full_data = vx.open('full_data_all_columns.hdf5')

In [6]:
route_717ADOWN_BR = full_data[full_data['ROUTENAME'] == '717ADOWN']

In [7]:
route_717ADOWN_BR

#,MESSAGE_ID,TRIP_ID,ROUTENAME,BUS_ID,ROUTE_ORIGIN_STOP,ROUTE_DESTINATION_STOP,TRIP_START_TIME,TRIP_END_TIME,AMOUNT,USER_COUNT,TICKET DATE,TICKET TIME,Tareek,date_time,Day,Hour,Month,day_type,Period
0,12434010719071044,T12434010719063403,717ADOWN,DL1PC6367,Andheria Bagh More,Tughlaqabad Village,01-JUL-2019 06:34:17,01-JUL-2019 07:50:41,10,1,01/07/2019,07:10:37,01/07/2019 07:10:37,2019-07-01 07:10:37,Monday,7,7,Weekday,July-Oct
1,12434010719071045,T12434010719063403,717ADOWN,DL1PC6367,Andheria Bagh More,Badarpur Border,01-JUL-2019 06:34:17,01-JUL-2019 07:50:41,15,1,01/07/2019,07:10:57,01/07/2019 07:10:57,2019-07-01 07:10:57,Monday,7,7,Weekday,July-Oct
2,12434010719064546,T12434010719063403,717ADOWN,DL1PC6367,Shiv Murti,Badarpur Border,01-JUL-2019 06:34:17,01-JUL-2019 07:50:41,15,1,01/07/2019,06:45:15,01/07/2019 06:45:15,2019-07-01 06:45:15,Monday,6,7,Weekday,July-Oct
3,12434010719064547,T12434010719063403,717ADOWN,DL1PC6367,Shiv Murti,Badarpur Border,01-JUL-2019 06:34:17,01-JUL-2019 07:50:41,15,1,01/07/2019,06:45:26,01/07/2019 06:45:26,2019-07-01 06:45:26,Monday,6,7,Weekday,July-Oct
4,12434010719064548,T12434010719063403,717ADOWN,DL1PC6367,Shiv Murti,Lado Sarai,01-JUL-2019 06:34:17,01-JUL-2019 07:50:41,10,1,01/07/2019,06:45:33,01/07/2019 06:45:33,2019-07-01 06:45:33,Monday,6,7,Weekday,July-Oct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"2,022,038",11333291219095232,T11333291219085003,717ADOWN,DL1PC6350,Hamdard Nagar / Sangam Vihar,Badarpur Border,29-DEC-2019 08:50:36,29-DEC-2019 10:12:20,10,1,29/12/2019,09:52:30,29/12/2019 09:52:30,2019-12-29 09:52:30,Sunday,9,12,Weekend,Nov-Dec
"2,022,039",12406291219231229,T12406291219222204,717ADOWN,DL1P6545,Ambedkar Nagar Terminal,Badarpur Border,29-DEC-2019 22:22:05,29-DEC-2019 23:22:52,10,1,29/12/2019,23:12:16,29/12/2019 23:12:16,2019-12-29 23:12:16,Sunday,23,12,Weekend,Nov-Dec
"2,022,040",11311291219181707,T11311291219172502,717ADOWN,DL1PC 6357,Lado Sarai,Prehlad Pur,29-DEC-2019 17:25:42,29-DEC-2019 18:56:51,10,1,29/12/2019,18:17:05,29/12/2019 18:17:05,2019-12-29 18:17:05,Sunday,18,12,Weekend,Nov-Dec
"2,022,041",11311291219182024,T11311291219172502,717ADOWN,DL1PC 6357,Lado Sarai Crossing,Hamdard Nagar / Sangam Vihar,29-DEC-2019 17:25:42,29-DEC-2019 18:56:51,5,1,29/12/2019,18:20:06,29/12/2019 18:20:06,2019-12-29 18:20:06,Sunday,18,12,Weekend,Nov-Dec


In [12]:
initial_alight_stops = route_717ADOWN_BR['ROUTE_ORIGIN_STOP'].unique()

In [17]:
d = {}

for c in route_717ADOWN_BR.column_names:
    d[c] = route_717ADOWN_BR[c].values
    
df = pd.DataFrame(d)
df

,MESSAGE_ID,TRIP_ID,ROUTENAME,BUS_ID,ROUTE_ORIGIN_STOP,ROUTE_DESTINATION_STOP,TRIP_START_TIME,TRIP_END_TIME,AMOUNT,USER_COUNT,TICKET DATE,TICKET TIME,Tareek,date_time,Day,Hour,Month,day_type,Period
0,12434010719071044,T12434010719063403,717ADOWN,DL1PC6367,Andheria Bagh More,Tughlaqabad Village,01-JUL-2019 06:34:17,01-JUL-2019 07:50:41,10,1,01/07/2019,07:10:37,01/07/2019 07:10:37,2019-07-01 07:10:37,Monday,7,7,Weekday,July-Oct
1,12434010719071045,T12434010719063403,717ADOWN,DL1PC6367,Andheria Bagh More,Badarpur Border,01-JUL-2019 06:34:17,01-JUL-2019 07:50:41,15,1,01/07/2019,07:10:57,01/07/2019 07:10:57,2019-07-01 07:10:57,Monday,7,7,Weekday,July-Oct
2,12434010719064546,T12434010719063403,717ADOWN,DL1PC6367,Shiv Murti,Badarpur Border,01-JUL-2019 06:34:17,01-JUL-2019 07:50:41,15,1,01/07/2019,06:45:15,01/07/2019 06:45:15,2019-07-01 06:45:15,Monday,6,7,Weekday,July-Oct
3,12434010719064547,T12434010719063403,717ADOWN,DL1PC6367,Shiv Murti,Badarpur Border,01-JUL-2019 06:34:17,01-JUL-2019 07:50:41,15,1,01/07/2019,06:45:26,01/07/2019 06:45:26,2019-07-01 06:45:26,Monday,6,7,Weekday,July-Oct
4,12434010719064548,T12434010719063403,717ADOWN,DL1PC6367,Shiv Murti,Lado Sarai,01-JUL-2019 06:34:17,01-JUL-2019 07:50:41,10,1,01/07/2019,06:45:33,01/07/2019 06:45:33,2019-07-01 06:45:33,Monday,6,7,Weekday,July-Oct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022038,11333291219095232,T11333291219085003,717ADOWN,DL1PC6350,Hamdard Nagar / Sangam Vihar,Badarpur Border,29-DEC-2019 08:50:36,29-DEC-2019 10:12:20,10,1,29/12/2019,09:52:30,29/12/2019 09:52:30,2019-12-29 09:52:30,Sunday,9,12,Weekend,Nov-Dec
2022039,12406291219231229,T12406291219222204,717ADOWN,DL1P6545,Ambedkar Nagar Terminal,Badarpur Border,29-DEC-2019 22:22:05,29-DEC-2019 23:22:52,10,1,29/12/2019,23:12:16,29/12/2019 23:12:16,2019-12-29 23:12:16,Sunday,23,12,Weekend,Nov-Dec
2022040,11311291219181707,T11311291219172502,717ADOWN,DL1PC 6357,Lado Sarai,Prehlad Pur,29-DEC-2019 17:25:42,29-DEC-2019 18:56:51,10,1,29/12/2019,18:17:05,29/12/2019 18:17:05,2019-12-29 18:17:05,Sunday,18,12,Weekend,Nov-Dec
2022041,11311291219182024,T11311291219172502,717ADOWN,DL1PC 6357,Lado Sarai Crossing,Hamdard Nagar / Sangam Vihar,29-DEC-2019 17:25:42,29-DEC-2019 18:56:51,5,1,29/12/2019,18:20:06,29/12/2019 18:20:06,2019-12-29 18:20:06,Sunday,18,12,Weekend,Nov-Dec


In [18]:
# df.to_csv('717ADOWN.csv',index=False)

In [19]:
scheduel = pd.read_csv("GTFS/stop_times.txt")
stops = pd.read_csv("GTFS/stops.txt")
trips = pd.read_csv("GTFS/trips.txt")
route = pd.read_csv("GTFS/routes.txt")

In [20]:
scheduel[['route_id', 'Hour', 'minute']] = scheduel['trip_id'].str.split('_', expand=True)

In [22]:
routename = '717ADOWN'

In [23]:
route[route["route_long_name"] == routename]

,route_short_name,route_long_name,route_type,route_id,agency_id
954,NaN,717ADOWN,3,954,DIMTS


In [24]:
routeid = route[route["route_long_name"] == routename].route_id.values[0]
route[route["route_long_name"] == routename]
print("Route id is = ",routeid)

Route id is =  954


In [25]:
bus_trip = scheduel[scheduel["route_id"]==str(routeid)]
bus_trip

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,route_id,Hour,minute
1357607,954_06_30,06:30:00,06:30:00,587,0,954,06,30
1357608,954_06_30,06:31:01,06:31:01,588,1,954,06,30
1357609,954_06_30,06:34:30,06:34:30,589,2,954,06,30
1357610,954_06_30,06:36:14,06:36:14,590,3,954,06,30
1357611,954_06_30,06:38:08,06:38:08,1259,4,954,06,30
...,...,...,...,...,...,...,...,...
1362306,954_22_20,23:59:22,23:59:22,479,44,954,22,20
1362307,954_22_20,24:00:58,24:00:58,480,45,954,22,20
1362308,954_22_20,24:02:48,24:02:48,481,46,954,22,20
1362309,954_22_20,24:04:28,24:04:28,482,47,954,22,20


In [26]:
stopsequence = [i for i in bus_trip[bus_trip["trip_id"]==bus_trip["trip_id"].values[0]]["stop_id"].values]

In [27]:
stop_order = pd.DataFrame(columns=stops.columns)
for i in stopsequence:
    a = stops[stops["stop_id"]==i].values[0]
    stop_order.loc[len(stop_order.index)] = a

trip_stops_names = stop_order['stop_name'].values
trip_stops_names

array(['Kapashera Border', 'Kapashera Village', 'Kapashera Crossing',
       'Samalkha Village', 'Samalkha Crossing', 'Shiv Murti', 'Rangpuri',
       'Mahipal Pur Crossing', 'Mahipal Pur Village', 'E-2 Vasant Kunj',
       'Rang Puri Pahari', 'Vasant Kunj Sec-C-8',
       'ISIC Hospital (Verma farm)',
       'Kathuria Public School (Sai Darbar Mandir)', 'Masood Pur Village',
       'Vasant Kunj sec-B1', 'Vasant Kunj Sec-D/3-4', 'Kishan Garh',
       'Vasant Kunj Sec-A ILBS', 'Vasant Kunj Sec-A Pkt-BC',
       'Andheria More / Chhatarpur Metro Station', 'Andheria Bagh More',
       'Qutub Minar Metro Sation', 'Ahinsa Sthal', 'Lado Sarai Crossing',
       'Lado Sarai', 'Saidul-A-Jab / Saket Metro Station',
       'Maidan Garhi Crossing (SDM Court)', 'Saket Crossing',
       'Asian Market', 'DIPSAR College of Pharmacy',
       'Ambedkar Nagar Terminal', 'Ambedkar Nagar Depot',
       'Khanpur Extension / Devli Xing', 'Vayu Sena Bad (Tigri)',
       'Batra Hospital / Satya Narayan Mandir'

In [29]:
route_origin_etm = route_717ADOWN_BR["ROUTE_ORIGIN_STOP"].unique()
route_origin_etm

['Ambedkar Nagar Terminal',
 'Asian Market',
 'Qutub Minar Metro Sation',
 'Andheria Bagh More',
 'Mahipal Pur Village',
 'Samalkha Village',
 'Masood Pur Village',
 'Andheria More / Chhatarpur Metro Station',
 'Vasant Kunj Sec-A Pkt-BC',
 'Surajkund Crossing',
 'E-2 Vasant Kunj',
 'Prehlad Pur',
 'Kapashera Border',
 'Badarpur M B Road',
 'Kaya Maya Hospital',
 'Ahinsa Sthal',
 'Kishan Garh',
 'Lado Sarai',
 'Maidan Garhi Crossing (SDM Court)',
 'Tughlaqabad Village',
 'Hamdard Nagar / Sangam Vihar',
 'Lado Sarai Crossing',
 'Shiv Murti',
 'Kapashera Crossing']

In [30]:
trip_stops_names = list(trip_stops_names)
trip_stops_names

['Kapashera Border',
 'Kapashera Village',
 'Kapashera Crossing',
 'Samalkha Village',
 'Samalkha Crossing',
 'Shiv Murti',
 'Rangpuri',
 'Mahipal Pur Crossing',
 'Mahipal Pur Village',
 'E-2 Vasant Kunj',
 'Rang Puri Pahari',
 'Vasant Kunj Sec-C-8',
 'ISIC Hospital (Verma farm)',
 'Kathuria Public School (Sai Darbar Mandir)',
 'Masood Pur Village',
 'Vasant Kunj sec-B1',
 'Vasant Kunj Sec-D/3-4',
 'Kishan Garh',
 'Vasant Kunj Sec-A ILBS',
 'Vasant Kunj Sec-A Pkt-BC',
 'Andheria More / Chhatarpur Metro Station',
 'Andheria Bagh More',
 'Qutub Minar Metro Sation',
 'Ahinsa Sthal',
 'Lado Sarai Crossing',
 'Lado Sarai',
 'Saidul-A-Jab / Saket Metro Station',
 'Maidan Garhi Crossing (SDM Court)',
 'Saket Crossing',
 'Asian Market',
 'DIPSAR College of Pharmacy',
 'Ambedkar Nagar Terminal',
 'Ambedkar Nagar Depot',
 'Khanpur Extension / Devli Xing',
 'Vayu Sena Bad (Tigri)',
 'Batra Hospital / Satya Narayan Mandir',
 'Hamdard Nagar / Sangam Vihar',
 'Air Force Station',
 'Air Force Station

In [42]:
boarding_in_correct_order = []

for s in trip_stops_names:
    if s in route_origin_etm:
        boarding_in_correct_order.append(s)
        
boarding_in_correct_order

['Kapashera Border',
 'Kapashera Crossing',
 'Samalkha Village',
 'Shiv Murti',
 'Mahipal Pur Village',
 'E-2 Vasant Kunj',
 'Masood Pur Village',
 'Kishan Garh',
 'Vasant Kunj Sec-A Pkt-BC',
 'Andheria More / Chhatarpur Metro Station',
 'Andheria Bagh More',
 'Qutub Minar Metro Sation',
 'Ahinsa Sthal',
 'Lado Sarai Crossing',
 'Lado Sarai',
 'Maidan Garhi Crossing (SDM Court)',
 'Asian Market',
 'Ambedkar Nagar Terminal',
 'Hamdard Nagar / Sangam Vihar',
 'Tughlaqabad Village',
 'Kaya Maya Hospital',
 'Surajkund Crossing',
 'Prehlad Pur',
 'Badarpur M B Road']

In [33]:
rd_717AUP = pd.read_csv('Redistributed_717AUP.csv')

In [34]:
rd_717AUP

,MESSAGE_ID,TRIP_ID,ROUTENAME,BUS_ID,ROUTE_ORIGIN_STOP,ROUTE_DESTINATION_STOP,TRIP_START_TIME,TRIP_END_TIME,AMOUNT,USER_COUNT,TICKET DATE,TICKET TIME,date_time,ticket_date,Day,Hour,Month,Origin_cordinates,Dest_cordinates
0,12695010919065107,T12695010919061902,717AUP,DL1PC6341,Andheria Bagh More,Kapashera Border,01-SEP-2019 06:19:46,01-SEP-2019 07:19:28,15,1,01/09/2019,06:51:55,2019-09-01 06:51:55,2019-09-01,Sunday,6,9,"(28.509217, 77.178967)","(28.522481, 77.082442)"
1,12420010919084834,T12420010919074802,717AUP,DL1PC6354,Andheria Bagh More / Chhatarpur Metro Station,Kapashera Border,01-SEP-2019 07:48:00,01-SEP-2019 09:26:48,15,1,01/09/2019,08:48:17,2019-09-01 08:48:17,2019-09-01,Sunday,8,9,"(28.509217, 77.178967)","(28.522481, 77.082442)"
2,12420010919084835,T12420010919074802,717AUP,DL1PC6354,Andheria Bagh More / Chhatarpur Metro Station,Kapashera Border,01-SEP-2019 07:48:00,01-SEP-2019 09:26:48,15,1,01/09/2019,08:48:19,2019-09-01 08:48:19,2019-09-01,Sunday,8,9,"(28.509217, 77.178967)","(28.522481, 77.082442)"
3,12678010919095238,T12678010919084802,717AUP,DL1PC6308,Andheria Bagh More / Chhatarpur Metro Station,Kapashera Border,01-SEP-2019 08:48:17,01-SEP-2019 10:34:02,15,1,01/09/2019,09:52:32,2019-09-01 09:52:32,2019-09-01,Sunday,9,9,"(28.509217, 77.178967)","(28.522481, 77.082442)"
4,10592010919093428,T10592010919083602,717AUP,DL1P6359,Andheria Bagh More / Chhatarpur Metro Station,Kapashera Border,01-SEP-2019 08:36:05,01-SEP-2019 10:10:08,15,1,01/09/2019,09:34:44,2019-09-01 09:34:44,2019-09-01,Sunday,9,9,"(28.509217, 77.178967)","(28.522481, 77.082442)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1393478,11333291219103256,T11333291219102504,717AUP,DL1PC6350,Jaitpur Crossing,Kapashera Border,29-DEC-2019 10:25:06,29-DEC-2019 11:51:02,15,1,29/12/2019,10:32:27,2019-12-29 10:32:27,2019-12-29,Sunday,10,12,"(28.4927, 77.30381700000002)","(28.522481, 77.082442)"
1393479,11333291219103257,T11333291219102504,717AUP,DL1PC6350,Jaitpur Crossing,Tughalqabad Village,29-DEC-2019 10:25:06,29-DEC-2019 11:51:02,5,1,29/12/2019,10:32:31,2019-12-29 10:32:31,2019-12-29,Sunday,10,12,"(28.4927, 77.30381700000002)","(28.509067, 77.265117)"
1393480,11311291219201613,T11311291219185703,717AUP,DL1PC 6357,Hamdard Nagar / Sangam Vihar,Kapashera Border,29-DEC-2019 18:57:26,29-DEC-2019 21:17:04,15,1,29/12/2019,20:16:45,2019-12-29 20:16:45,2019-12-29,Sunday,20,12,"(28.512167, 77.248667)","(28.522481, 77.082442)"
1393481,11311291219202124,T11311291219185703,717AUP,DL1PC 6357,Batra Hospital (Satya Narayan Mandir),Kishan Garh,29-DEC-2019 18:57:26,29-DEC-2019 21:17:04,10,1,29/12/2019,20:21:08,2019-12-29 20:21:08,2019-12-29,Sunday,20,12,"(28.512167, 77.248667)","(28.514366, 77.16415500000002)"


In [37]:
routeid = route[route["route_long_name"] == '717AUP'].route_id.values[0]
route[route["route_long_name"] == routename]
print("Route id is = ",routeid)

Route id is =  141


In [38]:
bus_trip = scheduel[scheduel["route_id"]==str(routeid)]

In [39]:
stopsequence = [i for i in bus_trip[bus_trip["trip_id"]==bus_trip["trip_id"].values[0]]["stop_id"].values]

In [64]:
stop_order = pd.DataFrame(columns=stops.columns)
for i in stopsequence:
    a = stops[stops["stop_id"]==i].values[0]
    stop_order.loc[len(stop_order.index)] = a

trip_stops_names_UP = stop_order['stop_name'].values
trip_stops_names_UP = list(trip_stops_names_UP)
trip_stops_names_UP

['Badarpur Border',
 'Jaitpur Crossing',
 'Badarpur M B Road / Rajiv Gandhi Stadium',
 'Prehlad Pur',
 'Surajkund Crossing',
 'Lal Kuan',
 'Prem Nagar',
 'Tuglaqabad MB Road',
 'Kaya Maya Hospital',
 'Tughalqabad Village',
 'Air Force Station',
 'Air Force Station',
 'Hamdard Nagar / Sangam Vihar',
 'Batra Hospital (Satya Narayan Mandir)',
 'Vayu Sena Bad (Tigri)',
 'Khanpur Extension / Devli Crossing',
 'Ambedkar Nagar Depot',
 'Ambedkar Nagar Terminal / DIPSAR',
 'Asian Market',
 'Saket Crossing',
 'Maidan Garhi Crossing (SDM Court)',
 'Saket Metro Sation',
 'Saidul-A-Jab',
 'Dhaula Peer',
 'Dhaula Peer / Lado Sarai',
 'Ahinsa Sthal',
 'Qutub Minar Metro Sation',
 'Andheria Bagh More',
 'Andheria Bagh More / Chhatarpur Metro Station',
 'Vasant Kunj Sec-A Pkt-BC',
 'Vasant Kunj Sec-A ILBS',
 'Kishan Garh',
 'Vasant Kunj Sec-D/3-4',
 'Vasant Kunj sec-B1',
 'Masood Pur Village',
 'Masood Pur Dairy',
 'Kathuria Public School (Sai Darbar Mandir)',
 'ISIC Hospital (Verma farm)',
 'Vasant K

In [43]:
alight_in_correct_order = []
for s in trip_stops_names_UP:
    if s in rd_717AUP['ROUTE_DESTINATION_STOP'].unique():
            alight_in_correct_order.append(s)
        
alight_in_correct_order

['Prehlad Pur',
 'Tughalqabad Village',
 'Hamdard Nagar / Sangam Vihar',
 'Ambedkar Nagar Terminal / DIPSAR',
 'Saidul-A-Jab',
 'Dhaula Peer / Lado Sarai',
 'Andheria Bagh More',
 'Kishan Garh',
 'ISIC Hospital (Verma farm)',
 'Mahipal Pur Village',
 'Mahipal Pur Village',
 'Shiv Murti',
 'Rajokari Crossing',
 'Shiv Murti / Dwarka More',
 'Samalkha Village',
 'Kapashera Crossing',
 'Kapashera Border']

In [ ]:
'Kapashera Border',
 'Kapashera Crossing',
 'Samalkha Village',
 'Shiv Murti',
 'Mahipal Pur Village',
 'E-2 Vasant Kunj',
 'Masood Pur Village',
 'Kishan Garh',
 'Vasant Kunj Sec-A Pkt-BC',
 'Andheria More / Chhatarpur Metro Station',
 'Andheria Bagh More',
 'Qutub Minar Metro Sation',
 'Ahinsa Sthal',
 'Lado Sarai Crossing',
 'Lado Sarai',
 'Maidan Garhi Crossing (SDM Court)',
 'Asian Market',
 'Ambedkar Nagar Terminal',
 'Hamdard Nagar / Sangam Vihar',
 'Tughlaqabad Village',
 'Kaya Maya Hospital',
 'Surajkund Crossing',
 'Prehlad Pur',
 'Badarpur M B Road'

In [51]:
rd_717AUP_alight = rd_717AUP.groupby(by=['ROUTE_DESTINATION_STOP'])[['USER_COUNT']].sum()
rd_717AUP_alight.reset_index(inplace=True)
rd_717AUP_alight['USER_COUNT'] = rd_717AUP_alight['USER_COUNT']/rd_717AUP['TRIP_ID'].nunique()
rd_717AUP_alight

,ROUTE_DESTINATION_STOP,USER_COUNT
0,Ambedkar Nagar Terminal / DIPSAR,1.186509
1,Andheria Bagh More,2.056775
2,Dhaula Peer / Lado Sarai,6.484314
3,Hamdard Nagar / Sangam Vihar,3.243815
4,ISIC Hospital (Verma farm),7.292631
5,Kapashera Border,30.041719
6,Kapashera Crossing,2.564502
7,Kishan Garh,4.998077
8,Mahipal Pur Village,7.826557
9,Prehlad Pur,0.001128


In [60]:
rd_717AUP_alight_d = {}
for i in range(rd_717AUP_alight.shape[0]):
    rd_717AUP_alight_d[rd_717AUP_alight.iloc[i,0]] = math.floor(rd_717AUP_alight.iloc[i,1])
rd_717AUP_alight_d

{'Ambedkar Nagar Terminal / DIPSAR': 1,
 'Andheria Bagh More': 2,
 'Dhaula Peer / Lado Sarai': 6,
 'Hamdard Nagar / Sangam Vihar': 3,
 'ISIC Hospital (Verma farm)': 7,
 'Kapashera Border': 30,
 'Kapashera Crossing': 2,
 'Kishan Garh': 4,
 'Mahipal Pur Village': 7,
 'Prehlad Pur': 0,
 'Rajokari Crossing': 5,
 'Saidul-A-Jab': 12,
 'Samalkha Village': 1,
 'Shiv Murti': 4,
 'Shiv Murti / Dwarka More': 0,
 'Tughalqabad Village': 2}

In [61]:
alight_in_correct_order

['Prehlad Pur',
 'Tughalqabad Village',
 'Hamdard Nagar / Sangam Vihar',
 'Ambedkar Nagar Terminal / DIPSAR',
 'Saidul-A-Jab',
 'Dhaula Peer / Lado Sarai',
 'Andheria Bagh More',
 'Kishan Garh',
 'ISIC Hospital (Verma farm)',
 'Mahipal Pur Village',
 'Mahipal Pur Village',
 'Shiv Murti',
 'Rajokari Crossing',
 'Shiv Murti / Dwarka More',
 'Samalkha Village',
 'Kapashera Crossing',
 'Kapashera Border']

In [65]:
trip_stops_names_UP

['Badarpur Border',
 'Jaitpur Crossing',
 'Badarpur M B Road / Rajiv Gandhi Stadium',
 'Prehlad Pur',
 'Surajkund Crossing',
 'Lal Kuan',
 'Prem Nagar',
 'Tuglaqabad MB Road',
 'Kaya Maya Hospital',
 'Tughalqabad Village',
 'Air Force Station',
 'Air Force Station',
 'Hamdard Nagar / Sangam Vihar',
 'Batra Hospital (Satya Narayan Mandir)',
 'Vayu Sena Bad (Tigri)',
 'Khanpur Extension / Devli Crossing',
 'Ambedkar Nagar Depot',
 'Ambedkar Nagar Terminal / DIPSAR',
 'Asian Market',
 'Saket Crossing',
 'Maidan Garhi Crossing (SDM Court)',
 'Saket Metro Sation',
 'Saidul-A-Jab',
 'Dhaula Peer',
 'Dhaula Peer / Lado Sarai',
 'Ahinsa Sthal',
 'Qutub Minar Metro Sation',
 'Andheria Bagh More',
 'Andheria Bagh More / Chhatarpur Metro Station',
 'Vasant Kunj Sec-A Pkt-BC',
 'Vasant Kunj Sec-A ILBS',
 'Kishan Garh',
 'Vasant Kunj Sec-D/3-4',
 'Vasant Kunj sec-B1',
 'Masood Pur Village',
 'Masood Pur Dairy',
 'Kathuria Public School (Sai Darbar Mandir)',
 'ISIC Hospital (Verma farm)',
 'Vasant K

In [ ]:
## find array of ratio of distribution

In [93]:
RD_717ADOWN = pd.read_csv('Redistributed_717ADOWN.csv')
RD_717ADOWN_grp = RD_717ADOWN.groupby(by='ROUTE_ORIGIN_STOP')[['USER_COUNT']].sum()
RD_717ADOWN_grp.reset_index(inplace=True)
RD_717ADOWN_grp

,ROUTE_ORIGIN_STOP,USER_COUNT
0,Ahinsa Sthal,1436
1,Air Force Station,37986
2,Ambedkar Nagar Depot,112627
3,Ambedkar Nagar Terminal,61486
4,Andheria Bagh More,140980
5,Andheria More / Chhatarpur Metro Station,544
6,Asian Market,7400
7,Badarpur M B Road,1
8,Batra Hospital / Satya Narayan Mandir,4552
9,DIPSAR College of Pharmacy,255


In [102]:
P_count = []

for s in trip_stops_names_UP:
    try:
        P_count.append(RD_717ADOWN_grp[RD_717ADOWN_grp['ROUTE_ORIGIN_STOP'] == s]['USER_COUNT'].values[0])
    except:
        P_count.append(0)
    
alight_ratio = pd.DataFrame(list(zip(trip_stops_names_UP, P_count)),columns=['Destination','User_count'])

In [103]:
alight_ratio

,Destination,User_count
0,Badarpur Border,0
1,Jaitpur Crossing,0
2,Badarpur M B Road / Rajiv Gandhi Stadium,0
3,Prehlad Pur,425
4,Surajkund Crossing,8
5,Lal Kuan,18
6,Prem Nagar,260
7,Tuglaqabad MB Road,0
8,Kaya Maya Hospital,20892
9,Tughalqabad Village,0


In [104]:
alight_in_correct_order

['Prehlad Pur',
 'Tughalqabad Village',
 'Hamdard Nagar / Sangam Vihar',
 'Ambedkar Nagar Terminal / DIPSAR',
 'Saidul-A-Jab',
 'Dhaula Peer / Lado Sarai',
 'Andheria Bagh More',
 'Kishan Garh',
 'ISIC Hospital (Verma farm)',
 'Mahipal Pur Village',
 'Mahipal Pur Village',
 'Shiv Murti',
 'Rajokari Crossing',
 'Shiv Murti / Dwarka More',
 'Samalkha Village',
 'Kapashera Crossing',
 'Kapashera Border']

In [138]:
def convert_to_percentage_ratio(numbers):
    total = sum(numbers)

    if total == 0:
        # handle the case where the total is zero
        return [0] * len(numbers)

    return [num / total for num in numbers]

def distribute_apples(n, weights):
    # calculate the total weight
    total_weight = sum(weights)

    if total_weight == 0:
        # handle the case where the total weight is zero
        return [0] * len(weights)
    
    # calculate the number of apples to give to each person based on their weight percentage
    apple_counts = [int(n * weight / total_weight) for weight in weights]

    # calculate the remaining apples to be distributed based on decimal values
    remaining_apples = n - sum(apple_counts)
    decimals = [(i, weight * remaining_apples % 1) for i, weight in enumerate(weights)]
    decimals = sorted(decimals, key=lambda x: x[1], reverse=True)

    # distribute the remaining apples based on the decimal values
    for i in range(remaining_apples):
        index = decimals[i][0]
        apple_counts[index] += 1

    return apple_counts




final_alight = {}
for stop in alight_in_correct_order:
    temp_list = []
    temp_user_count = []
    for s in trip_stops_names_UP:
        if(s in final_alight.keys()):
            continue
        else:
            if (s == stop):
                temp_list.append(s)
                temp_user_count.append(alight_ratio[alight_ratio['Destination'] == s]['User_count'].values[0])

                temp_user_count = convert_to_percentage_ratio(temp_user_count)
                redistributed_value = distribute_apples(rd_717AUP_alight_d[stop],temp_user_count)

                temp_dict = {name:value for name,value in zip(temp_list,redistributed_value)}
                final_alight.update(temp_dict)

                temp_list = []
                temp_user_count = []

                break
            else:
                temp_list.append(s)
                temp_user_count.append(alight_ratio[alight_ratio['Destination'] == s]['User_count'].values[0])
        

In [139]:
final_alight

{'Badarpur Border': 0,
 'Jaitpur Crossing': 0,
 'Badarpur M B Road / Rajiv Gandhi Stadium': 0,
 'Prehlad Pur': 0,
 'Surajkund Crossing': 0,
 'Lal Kuan': 0,
 'Prem Nagar': 0,
 'Tuglaqabad MB Road': 0,
 'Kaya Maya Hospital': 2,
 'Tughalqabad Village': 0,
 'Air Force Station': 1,
 'Hamdard Nagar / Sangam Vihar': 1,
 'Batra Hospital (Satya Narayan Mandir)': 0,
 'Vayu Sena Bad (Tigri)': 0,
 'Khanpur Extension / Devli Crossing': 0,
 'Ambedkar Nagar Depot': 1,
 'Ambedkar Nagar Terminal / DIPSAR': 0,
 'Asian Market': 8,
 'Saket Crossing': 1,
 'Maidan Garhi Crossing (SDM Court)': 3,
 'Saket Metro Sation': 0,
 'Saidul-A-Jab': 0,
 'Dhaula Peer': 0,
 'Dhaula Peer / Lado Sarai': 0,
 'Ahinsa Sthal': 0,
 'Qutub Minar Metro Sation': 0,
 'Andheria Bagh More': 2,
 'Andheria Bagh More / Chhatarpur Metro Station': 0,
 'Vasant Kunj Sec-A Pkt-BC': 0,
 'Vasant Kunj Sec-A ILBS': 1,
 'Kishan Garh': 3,
 'Vasant Kunj Sec-D/3-4': 0,
 'Vasant Kunj sec-B1': 1,
 'Masood Pur Village': 0,
 'Masood Pur Dairy': 0,
 'Kat

In [140]:
a = final_alight.keys()
b = final_alight.values()
final_alight_pd = pd.DataFrame(list(zip(a,b)),columns=['Destination','User_count'])
final_alight_pd

,Destination,User_count
0,Badarpur Border,0
1,Jaitpur Crossing,0
2,Badarpur M B Road / Rajiv Gandhi Stadium,0
3,Prehlad Pur,0
4,Surajkund Crossing,0
5,Lal Kuan,0
6,Prem Nagar,0
7,Tuglaqabad MB Road,0
8,Kaya Maya Hospital,2
9,Tughalqabad Village,0


In [142]:
def lat_finder(stop_name):
    lat = stops[(stops['stop_name']==stop_name)]['stop_lat'].iloc[0]
    return lat

def long_finder(stop_name):
    long = stops[(stops['stop_name']==stop_name)]['stop_lon'].iloc[0]
    return long

In [143]:
final_alight_pd['lat'] = final_alight_pd['Destination'].apply(lat_finder)
final_alight_pd['long'] = final_alight_pd['Destination'].apply(long_finder)

In [144]:
final_alight_pd

,Destination,User_count,lat,long
0,Badarpur Border,0,28.492700,77.303817
1,Jaitpur Crossing,0,28.497217,77.302647
2,Badarpur M B Road / Rajiv Gandhi Stadium,0,28.499178,77.299004
3,Prehlad Pur,0,28.500000,77.294633
4,Surajkund Crossing,0,28.501700,77.291000
5,Lal Kuan,0,28.502600,77.284517
6,Prem Nagar,0,28.742883,77.146467
7,Tuglaqabad MB Road,0,28.505533,77.276483
8,Kaya Maya Hospital,2,28.508250,77.269017
9,Tughalqabad Village,0,28.509067,77.265117


In [151]:
final_alight_pd.to_csv('alight_717AUP_rd.csv',index=False)

In [148]:
rd_717AUP_alight['lat'] = rd_717AUP_alight['ROUTE_DESTINATION_STOP'].apply(lat_finder)
rd_717AUP_alight['long'] = rd_717AUP_alight['ROUTE_DESTINATION_STOP'].apply(long_finder)

In [150]:
rd_717AUP_alight.to_csv('alight_717AUP.csv',index=False)